In [1]:
!pip install hyperopt

In [22]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from skimage import color, exposure
from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe,  Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
ERROR! Session/line number was not unique in database. History logging moved to new session 65


In [6]:
cd '/content/drive/My Drive/Colab Notebooks/road_sign/data/road_sine_ml/data'

/content/drive/My Drive/Colab Notebooks/road_sign/data/road_sine_ml/data


In [0]:
train = pd.read_pickle('train.p')
test = pd.read_pickle('test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1:
  y_train = to_categorical(y_train)

if y_test.ndim == 1:
  y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir= os.path.join("logs", datetime.datetime.now().strftime("%Ym%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epchos', 5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback])
  
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def gen_cnn_v5(input_shape, num_classes):
  return Sequential([
                    Conv2D(
                        filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape),
                     Conv2D(
                        filters=32,
                        kernel_size=(3, 3),
                        activation='relu'),
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(
                        filters=62,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape,
                        padding='same'),
                     Conv2D(
                        filters=64,
                        kernel_size=(3, 3),
                        activation='relu'),
                     
                     MaxPool2D(),
                     Dropout(0.3),

                     Conv2D(
                        filters=62,
                        kernel_size=(3, 3),
                        activation='relu',
                        padding='same'),
                     Conv2D(
                        filters=62,
                        kernel_size=(3, 3),
                        activation='relu'),
                     
                     MaxPool2D(),
                     Dropout(0.3),
                     
                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),

                     Dense(1024, activation='relu'),
                     Dropout(0.3),
                     
                     Dense(num_classes, activation='softmax')

  ])

In [12]:
model = gen_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 5s 18ms/step - loss: 2.4552 - accuracy: 0.3263 - val_loss: 0.7628 - val_accuracy: 0.7800
Epoch 2/5
272/272 [==============================] - 4s 16ms/step - loss: 0.7268 - accuracy: 0.7765 - val_loss: 0.1656 - val_accuracy: 0.9529
Epoch 3/5
272/272 [==============================] - 4s 16ms/step - loss: 0.3512 - accuracy: 0.8958 - val_loss: 0.0815 - val_accuracy: 0.9771
Epoch 4/5
272/272 [==============================] - 4s 16ms/step - loss: 0.2377 - accuracy: 0.9293 - val_loss: 0.0443 - val_accuracy: 0.9879
Epoch 5/5
272/272 [==============================] - 4s 16ms/step - loss: 0.1851 - accuracy: 0.9473 - val_loss: 0.0266 - val_accuracy: 0.9926


0.9834467120181406

In [0]:
def get_model(params):
  return Sequential([
                    Conv2D(
                        filters=32,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape),
                     Conv2D(
                        filters=32,
                        kernel_size=(3, 3),
                        activation='relu'),
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_one']),

                     Conv2D(
                        filters=62,
                        kernel_size=(3, 3),
                        activation='relu',
                        input_shape=input_shape,
                        padding='same'),
                     Conv2D(
                        filters=62,
                        kernel_size=(3, 3),
                        activation='relu'),
                     
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_two']),

                     Conv2D(
                        filters=128,
                        kernel_size=(3, 3),
                        activation='relu',
                        padding='same'),
                     Conv2D(
                        filters=128,
                        kernel_size=(3, 3),
                        activation='relu'),
                     
                     MaxPool2D(),
                     Dropout(params['dropout_cnn_block_three']),
                     
                     Flatten(),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_one']),

                     Dense(1024, activation='relu'),
                     Dropout(params['dropout_dense_block_two']),
                     
                     Dense(num_classes, activation='softmax')

  ])

In [17]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.0799 - accuracy: 0.9834


[0.07986006140708923, 0.9834467172622681]

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size= 128, #int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
  )

  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))
  
  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [51]:
space = {
    'batch_size': hp.quniform('batch_size', 50, 200, 20),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.6),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.6),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.6),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.6),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5206591526455211, 'dropout_cnn_block_three': 0.4524266287408073, 'dropout_cnn_block_two': 0.34853447365060797, 'dropout_dense_block_one': 0.5549758524090633, 'dropout_dense_block_two': 0.4445991415224857}
accuracy=0.9256235957145691
{'batch_size': 120.0, 'dropout_cnn_block_one': 0.3965681018633337, 'dropout_cnn_block_three': 0.5074192277850266, 'dropout_cnn_block_two': 0.4821427186215046, 'dropout_dense_block_one': 0.46061734122587045, 'dropout_dense_block_two': 0.47356612382842334}
accuracy=0.9052154421806335
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.5588374032499195, 'dropout_cnn_block_three': 0.5577388406647106, 'dropout_cnn_block_two': 0.5060235792430364, 'dropout_dense_block_one': 0.44079549219378905, 'dropout_dense_block_two': 0.37243929478262117}
accuracy=0.7319728136062622
{'batch_size': 80.0, 'dropout_cnn_block_one': 0.35580120645483765, 'dropout_cnn_block_three': 0.5019866346682017, 'dropout_cnn_block_two': 0.53692756089